In [1]:
# load matplotlib inline mode
%matplotlib inline

# import some useful libraries
import numpy as np                # numerical analysis linear algebra
import pandas as pd               # efficient tables
import matplotlib.pyplot as plt   # plotting
import matplotlib as matplotlib
from scipy import stats
from scipy.linalg import block_diag
import seaborn as sns

In [190]:
def _calc_D(s):
    # calc all the differences
    s0__1 = s[1:-1] - s[:-2]
    s1_0 = s[2:] - s[1:-1]
    s1__1 = s[2:] - s[:-2]

    # calc the -1, 0, and 1 diagonals
    A = -((s1_0/s1__1)/s0__1)
    B = -((s0__1/s1__1)/s1_0) + (s1_0/s1__1)/s0__1
    C = (s0__1/s1__1)/s1_0

    # create the matrix
    D = np.zeros((len(s), len(s)))
    D.flat[len(s):-2:len(s)+1] = A
    D.flat[len(s)+1:-1:len(s)+1] = B
    D.flat[len(s)+2::len(s)+1] = C
    
    return D.T
def my_mat_power(x, n, x0=None):
    if x0 is None:
        x0 = x
    if n == 1:
        return x
    elif n == 2:
        return np.dot(x, x0)
    elif n > 2:
        return my_mat_power(np.dot(x, x0), n-1, x0=x0)
    else:
        raise ValueError("Only positive n are allowed.")
    
def _calc_invL2(s, k):
    # calc finite differentiation matrix from s
    D = np.float64(_calc_D(np.float64(s)))
    
    # Post inverse Laplace linear operator
    #invL = (((-1.) ** k) / factorial(k)) * \
    #        np.dot(np.linalg.matrix_power(D, k), np.diag(s ** (k+1)))[:, k:-k]
    invL = (((-1.) ** k) * \
            my_mat_power(D, k) * np.exp(np.log(s)*(k+1) - 
                                        np.log(np.arange(2,k+1)).sum()))[:, k:-k]
    

    # return as ndarray
    return np.float64(invL)

In [247]:
class iSITH():
    def __init__(self, tau_min=.1, tau_max=20., k=8, ntau=50, dt=0.01, n_features=2, g=0.0, rho = [0]):
        
        self.k = k
        self.tau_min = tau_min
        self.tau_max = tau_max
        self.ntau = ntau
        self.ntau_k = ntau + 2*k
        self.dt = dt
        self.g = g
        self.rho = rho
        self.nf = n_features
        self.nr = len(rho)
        
        self.c = (self.tau_max/self.tau_min)**(1./(self.ntau-1))-1
        self.tau_star = self.tau_min * (1 + self.c)**np.arange(-self.k, self.ntau + self.k)
        self.s = (1/self.tau_star)
        self.invL = _calc_invL2(self.s, k)
        
        self.F_m = np.zeros([self.ntau_k,self.nf])
        self.M = np.zeros([self.nr,self.ntau_k,self.nf,self.nf])
        self.F_p = np.zeros([self.nr,self.ntau_k,self.nf])
        
    def present_stim(self, stim=None):
        
        if (stim is None) | (stim>=self.nf):
            return -1
        else:
            #generate stim vector
            stim_vec = np.zeros(self.nf)
            stim_vec[stim] = 1
            
            self.F_m += np.repeat(stim_vec[None,:], self.ntau_k, axis=0)
            self.F_p += np.tensordot(self.M, stim_vec, axes=([3],[0]))
            
            self.M[:,:,stim,:] = self.rho[:,None,None] * self.M[:,:,stim,:]
            long_stim = np.repeat(stim_vec[None,:], self.ntau_k, axis=0)
            
            self_prediction_mask = np.logical_not(np.outer(stim_vec,stim_vec))
            M_input = (self.F_m[:,np.newaxis,:] * long_stim[:,:,np.newaxis])*self_prediction_mask[None,:,:]
            self.M += ((1-self.rho)[:,None,None,None] * M_input[None,:,:,:])
    
    def evolve(self):
        self.F_m = self.F_m *  np.exp(-self.s[:,None]*self.dt)
        self.F_p = self.F_p *  np.exp(self.s[:,None]*self.dt)
        
        for i in range(self.nf):
            if self.F_p[0,-1,i] > self.F_p[0,-2,i]:
                self.F_p[0,:,i] = self.F_p[0,:,i] *  np.exp(-self.s*self.dt)
                F_smax = self.F_p[0,-1,i]
                self.F_p[0,:,i] -= F_smax
                
    def memory_wipe(self):
        self.F_m = np.zeros([self.ntau_k,self.nf])
        self.F_p = np.zeros([self.nr,self.ntau_k,self.nf])

In [241]:
tau_min = 0.1
tau_max = 10
k=8
ntau = 30
dt = 0.01
rho = np.array([0.4,0.6,0.8,0.9])

In [242]:
frasier = iSITH(tau_min=tau_min, tau_max=tau_max, k=k, ntau=ntau, dt=dt, rho=rho,n_features=2,g=0.0)

In [243]:
frasier.present_stim(stim=0)

In [244]:
steps = np.arange(0,4.0,dt)
for i in steps:
    frasier.evolve()
frasier.present_stim(stim=1)